## module

In [3]:
import json
from langchain.document_loaders import JSONLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

# # 1. JSON 파일을 불러와 문서로 변환
# train_med_json_directory = './dataset/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/02.라벨링데이터/Training_medical.json'
# train_leg_json_directory = './dataset/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/02.라벨링데이터/Training_legal.json'
# json_file_paths=[train_med_json_directory,train_leg_json_directory]
# json_datas = []
# for json_file_path in json_file_paths:
#     with open(json_file_path, 'r', encoding='utf-8') as file:
#         data = json.load(file)
#         json_datas.extend(data['data'])

import os
# JSON 파일 경로 설정
import json
import time
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### JSON File

In [4]:

# JSON 파일 경로 설정
train_med_json_directory = './dataset/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/02.라벨링데이터/Training_medical.json'
train_leg_json_directory = './dataset/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/02.라벨링데이터/Training_legal.json'
json_file_paths = [train_med_json_directory, train_leg_json_directory]

# 총 예상 소요시간과 현재 진행 퍼센트를 알 수 있게 하는 함수
def load_json_files(file_paths):
    json_datas = []
    total_files = len(file_paths)
    total_time = 0
    
    # 파일별로 진행 상황을 모니터링하기 위해 tqdm 사용
    for i, json_file_path in enumerate(tqdm(file_paths, desc="Loading JSON files", unit="file")):
        start_time = time.time()
        
        with open(json_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            json_datas.extend(data['data'])
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        total_time += elapsed_time
        print(f"File {i+1}/{total_files} processed in {elapsed_time:.2f} seconds")

    return json_datas, total_time

# JSON 파일 로드
json_datas, total_time = load_json_files(json_file_paths)

# 총 소요 시간과 진행 퍼센트 출력
print(f"Total time taken: {total_time:.2f} seconds")




Loading JSON files:  50%|█████     | 1/2 [00:10<00:10, 10.10s/file]

File 1/2 processed in 10.10 seconds


Loading JSON files:  50%|█████     | 1/2 [00:21<00:21, 21.10s/file]


KeyboardInterrupt: 

### Documenting

In [4]:
from langchain.schema import Document
from concurrent.futures import ThreadPoolExecutor

# 문서 생성
documents = [{"text": json.dumps(item)} for item in json_datas]
print("document ")
def split_document(doc):
    return text_splitter.split_text(doc["text"])

# 문서 나누기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = []

start_time = time.time()
with ThreadPoolExecutor() as executor:
    for split in tqdm(executor.map(split_document, documents), total=len(documents), desc="Splitting documents"):
        split_docs.extend(split)

end_time = time.time()
total_time = end_time - start_time
print(f"Total time taken for splitting: {total_time:.2f} seconds")
 
# Document 객체로 변환
document_objs = [Document(page_content=doc) for doc in split_docs]



document 


Splitting documents: 100%|██████████| 101211/101211 [00:01<00:00, 56788.61it/s]


Total time taken for splitting: 71.63 seconds


#### check documents

## model

### test1

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
# 데이터 샘플링 (예: 전체 데이터의 10%만 사용)
sample_size = int(len(json_datas) * 0.1)
sampled_data = json_datas[:sample_size]
print(f"Using a sample size of {sample_size} entries.")

# 문서 객체로 변환
document_objs = [Document(page_content=json.dumps(doc)) for doc in sampled_data]

# Embeddings 생성
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

# 병렬 처리 함수
def process_batch(batch):
    return Chroma.from_documents(documents=batch, embedding=embeddings)

# 배치 크기 설정
batch_size = 1000
batches = [document_objs[i:i + batch_size] for i in range(0, len(document_objs), batch_size)]

# 시작 시간 기록
start_time = time.time()

# 병렬 처리로 벡터 저장소 생성
vectordb_list = []
with ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_batch, batches), total=len(batches), desc="Creating vector store"))
    for result in results:
        vectordb_list.append(result)

# 종료 시간 기록
end_time = time.time()
total_time = end_time - start_time

# 총 소요 시간 출력
print(f"Total time taken for creating vector store: {total_time:.2f} seconds")

# 여러 벡터 저장소 병합 (필요한 경우)
# 이 부분은 Chroma에서 지원하지 않을 수 있으므로, 필요에 따라 다른 방법으로 병합하거나 별도로 처리할 수 있습니다.

# 예시 질문에 대한 답변 생성
retriever = vectordb_list[0].as_retriever()  # 예: 첫 번째 벡터 저장소 사용

Using a sample size of 10121 entries.


Creating vector store:   0%|          | 0/11 [00:00<?, ?it/s]


ValueError: Could not connect to tenant default_tenant. Are you sure it exists?

### test2

In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


template="""
db기반으로 대답해줘 {question}
{context}
"""
# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 질문에 대한 답변 생성
question = "김오순님의 건강 상태는 어떤가요?"
answer=rag_chain.run(question)
print(answer)
#여기서는 split_docs를 만들 때, 각 text를 분할하여 split_docs 리스트에 추가합니다. FAISS 벡터 저장소는 분할된 텍스트를 사용하여 인덱싱됩니다. 이후 RetrievalQA 체인을 사용하여 질문에 답변을 생성합니다.



KeyboardInterrupt: 

### test

#### json

In [2]:
import json
import time
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# JSON 파일 경로 설정
train_med_json_directory = './dataset/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/02.라벨링데이터/Training_medical.json'
train_leg_json_directory = './dataset/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/02.라벨링데이터/Training_legal.json'
json_file_paths = [train_med_json_directory, train_leg_json_directory]

# 총 예상 소요시간과 현재 진행 퍼센트를 알 수 있게 하는 함수
def load_json_files(file_paths):
    json_datas = []
    total_files = len(file_paths)
    total_time = 0

    # 파일별로 진행 상황을 모니터링하기 위해 tqdm 사용
    for i, json_file_path in enumerate(tqdm(file_paths, desc="Loading JSON files", unit="file")):
        start_time = time.time()

        with open(json_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            json_datas.extend(data['data'])

        end_time = time.time()
        elapsed_time = end_time - start_time
        total_time += elapsed_time
        print(f"File {i+1}/{total_files} processed in {elapsed_time:.2f} seconds")

    return json_datas, total_time

# JSON 파일 로드
json_datas, total_time = load_json_files(json_file_paths)

# 총 소요 시간 출력
print(f"Total time taken for loading JSON files: {total_time:.2f} seconds")

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# 문서 생성
documents = [{"text": item.get("text", "")} for item in json_datas]

# 문서 분할 함수
def split_document(doc):
    return text_splitter.split_text(doc["text"])

# 문서 분할
split_docs = []
start_time = time.time()
with ThreadPoolExecutor() as executor:
    for split in tqdm(executor.map(split_document, documents), total=len(documents), desc="Splitting documents"):
        split_docs.extend(split)
end_time = time.time()
total_time = end_time - start_time
print(f"Total time taken for splitting: {total_time:.2f} seconds")


Loading JSON files:  50%|█████     | 1/2 [00:07<00:07,  7.96s/file]

File 1/2 processed in 7.96 seconds


Loading JSON files:  50%|█████     | 1/2 [00:13<00:13, 13.60s/file]


KeyboardInterrupt: 

#### Text Files

In [4]:
import os
from glob import glob
import time
from concurrent.futures import ThreadPoolExecutor
import logging
from tqdm import tqdm
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
from openai import RateLimitError

# txt 파일 경로 설정
train_med_txt_directory = './dataset/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/01.원천데이터/TS_01.의료'
train_leg_txt_directory = './dataset/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/01.원천데이터/TS_02.법률'
txt_file_paths = [train_med_txt_directory, train_leg_txt_directory]

# PDF 및 텍스트 파일들을 로드하여 Document 객체로 변환
def load_documents(txt_file_paths):
    documents = []

    for path in txt_file_paths:
        text_files = glob(os.path.join(path, '*.txt'))
        for text_file in text_files:
            with open(text_file, 'r', encoding='utf-8') as file:
                text = file.read()
                # 텍스트를 Document 객체로 변환
                text_document = Document(page_content=text, metadata={"source": text_file})
                documents.append(text_document)

    return documents

# Document 객체를 로드
document_objs = load_documents(txt_file_paths)

# 텍스트를 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = chunk_splitter.split_documents(document_objs)

# 길이 제한 설정
max_len = 15000

# 문서 길이 제한 적용
for idx, doc in enumerate(split_docs):
    if len(doc.page_content) > max_len:
        print(idx)
        split_docs[idx].page_content = doc.page_content[:max_len]

#### Model

In [ ]:

# Embeddings 생성
# OPENAI_API_KEY = 'your_openai_api_key'  # 실제 API 키를 여기에 입력
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

# 재시도 가능한 벡터 저장소 생성 함수
def process_batch(batch):
    max_retries = 5
    delay = 5  # 지연 시간 (초)
    for attempt in range(max_retries):
        try:
            return Chroma.from_documents(documents=batch, embedding=embeddings)
        except RateLimitError as e:
            if attempt < max_retries - 1:
                logging.warning(f"RateLimitError: {e}. Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2  # 지연 시간을 지수적으로 증가
            else:
                logging.error(f"Failed after {max_retries} attempts.")
                raise

# 배치 크기 설정
batch_size = 250
batches = [document_objs[i:i + batch_size] for i in range(0, len(document_objs), batch_size)]

# 시작 시간 기록
start_time = time.time()

# 병렬 처리로 벡터 저장소 생성
vectordb_list = []
with ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_batch, batches), total=len(batches), desc="Creating vector store"))
    for result in results:
        vectordb_list.append(result)

# 종료 시간 기록
end_time = time.time()
total_time = end_time - start_time

# 총 소요 시간 출력
print(f"Total time taken for creating vector store: {total_time:.2f} seconds")

# 여러 벡터 저장소 병합 (필요한 경우)
# 이 부분은 Chroma에서 지원하지 않을 수 있으므로, 필요에 따라 다른 방법으로 병합하거나 별도로 처리할 수 있습니다.

# 예시 질문에 대한 답변 생성
retriever = vectordb_list[0].as_retriever()  # 예: 첫 번째 벡터 저장소 사용

# 예시 질문에 대한 답변 생성 코드 (필요 시)
query = "인공지능융합공학부에 대해서 알려줘"
results = retriever.retrieve(query)
print(results)